# Workshop Preperation for Classiq's Challenge - Quantum Arithmetics

Welcome to the Classiq challenge of the 2024 MIT IQuHack hackathon!
This Jupyter notebook is a tutorial workshop that should prepare you for the challenge itself.


Additional resources you should use are
- The IDE of the classiq platform at [platform.classiq.io](platform.classiq.io)
- The [community Slack of Classiq](https://short.classiq.io/join-slack) - Classiq's team will answer any question you have over there, including implementation questions
- [Classiq's documentation](https://docs.classiq.io/latest/user-guide/platform/) with the dedicated [Python SDK explanations](https://docs.classiq.io/latest/user-guide/platform/qmod/python/functions/)

Good luck!

![Classiq](MITClassiq.png)

## Setting The Scene

Install the Classiq SDK package:

In [1]:
!pip install -U classiq

You need to authenticate your device in order to use Classiq's backend synthesis engine and IDE. Make sure to register to the platform at [platform.classiq.io](platform.classiq.io) before you run the next cell:

In [2]:
import classiq
classiq.authenticate()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [3]:
from classiq import *

Some basic explanations about the high-level functional design with Classiq:

* There should always be a main (`def main(...)`) function - the model that captures your algortihm is described there

* The model is always generated out of the main function 

* The model is sent to the synthesis engine (compiler) that return a quantum program which contains the quantum circuit


Some basic guidelines about the modeling language (QMOD):

1. Every quantum variable should be declared, either as a parameter of a funciton e.g. `def prepare_minus(x: QBit)` or within the function itself with `x = QBit('x')`

2. Some quantum variables need to be initalized with the `allocate` function. This is required in 2 cases:
* A variable is a parameter of a function with the declaration `Output` like `def main(x: Output[QInt])`
* A variable that was declared within a function like `a = QInt('a')`

3. For the `main` function, you will always use `Output` for all variables, as the function does not receive any input

4. Every function you use with the QMOD language should have the decorator `@QFunc` before it

Important tip!

You can see all the declarations of the functions with what are their input arguments in the `functions.py` file within the classiq package (or by just right clicking a function and presing `Go To Defintion`)

Let's continue warming up with creating a function that receives a quantum register and creates a uniform superposistion for all qubits within this array. You should use the function `apply_to_all(gate_operand=, target=)`:

## Arithmetic Operations with Classiq

One of the key advantages of Classiq is it's simplistic and powerful compiler for quantum arithmetics. Let's see an example:

In [4]:
num_qubits = 1
fraction_digits = 1
is_signed = False

@QFunc
def main(x: Output[QNum], y: Output[QNum]):
    allocate_num(num_qubits=num_qubits, is_signed=is_signed, fraction_digits=fraction_digits, out=x)
    hadamard_transform(x)
    y|= x - ((1/3)*(x**3)) #+ ((1/15)*(2*(x**5))) #- 17*(x**7)/315

qmod = create_model(main)

The `allocate_num` function initalizes a quantum variable that represent numbers. By default it is initalized to the $\ket{0}$ state. Then the `hadmard_transform` create a superposition of all posible states in the domain $[-2^3,2^3-1]$. Finally, the arithmetic operation creates the entangled superpostion of states:
$\begin{equation}
\sum_{x =-2^3}^{2^3-1}\ket{x}\ket{x^2+1}
\end{equation}$

The `qmod` variable is a text file that captures the algortihm we have just created. Now, what we want is to synthesize (compile) in order to receive a concrete quantum program that contains the quantum circuit implementation.

In [5]:
qprog = synthesize(qmod)

And in order to view it:

In [6]:
show(qprog)

Opening: https://platform.classiq.io/circuit/ca980159-2590-4b00-a7f7-eb1d26834d9e?version=0.36.1


In [7]:
job = execute(qprog)

ClassiqAPIError: Call to API failed with code 400: Requested program requires too many qubits.
Requested qubits: 28, backend: aer_simulator, limit: 25.
Please run a different program or try a different backend.

If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack
If the error persists feel free to open a ticket at: https://short.classiq.io/support

In [ ]:
job.open_in_ide()